 Todo: Aktualisieren:
<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item">
    <li><span><a href="#|-Preliminaries" data-toc-modified-id="|-Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>| Preliminaries</a></span></li><li><span><a href="#|-Cleaning" data-toc-modified-id="|-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>| Cleaning</a></span></li><li><span><a href="#|-Named-Entity-Recognition-w/-spaCy" data-toc-modified-id="|-Named-Entity-Recognition-w/-spaCy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>| Named Entity Recognition w/ spaCy</a></span><ul class="toc-item"><li><span><a href="#Split-first,-middle-and-last-name-in-different-cols" data-toc-modified-id="Split-first,-middle-and-last-name-in-different-cols-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Split first, middle and last name in different cols</a></span></li><li><span><a href="#|-Enrich-first-name-if-only-last-name-is-mentioned" data-toc-modified-id="|-Enrich-first-name-if-only-last-name-is-mentioned-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>| Enrich first name if only last name is mentioned</a></span></li></ul></li><li><span><a href="#|-Gender-Guesser" data-toc-modified-id="|-Gender-Guesser-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>| Gender Guesser</a></span></li><li><span><a href="#|-Group-per-article-and-get-share-of-gender-per-article" data-toc-modified-id="|-Group-per-article-and-get-share-of-gender-per-article-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>| Group per article and get share of gender per article</a></span></li><li><span><a href="#|-Wikidata-Query" data-toc-modified-id="|-Wikidata-Query-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>| Wikidata Query</a></span></li></ul></div>

https://github.com/julesellaboranto/washington_post_bias

---
# | Preliminaries

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import regex as re
from pandarallel import pandarallel
import pathlib
import matplotlib.pyplot as plt

#from bs4 import BeautifulSoup as bs

# SpaCy 
import spacy
import en_core_web_trf
import en_core_web_md
import en_core_web_sm

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk

# TRF Orininal / Md schneller
nlp = spacy.load("en_core_web_md")
# JULIA nlp = spacy.load("en_core_web_trf")

from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

import warnings
warnings.filterwarnings('ignore') # (action='once')

import tqdm

import time

In [ ]:
pandarallel.initialize(progress_bar=True)
#pandarallel.initialize()

---
# | FILL OUTS

In [ ]:
myWishedDatasets = 100000
myWishedSmallFiles = 30
myTRECfile = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\TREC_Washington_Post_collection.v4.jl'
myZwischenSpeichern = False
myDataPath = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data'

zeitanfang = time.time()
print(f'Programmstart bei {myWishedDatasets} Datensätzen ----------------------------------------')

---
---
---
# | Pre Processing

In [ ]:
# Speichern des aktuellen Stands wenn gewollt
def zwischenSpeichern(path: str, version: str):
    zwischenstandPath = os.path.join(myDataPath, 'zwischenstand')
    MyPickleFile = f'zwischenstand_{version}.pickle'
    MyPickleFile = os.path.join(zwischenstandPath, MyPickleFile)
    
    if not os.path.exists(zwischenstandPath):
        os.makedirs(zwischenstandPath)
        print(f'{zwischenstandPath} wurde erstellt.')
    
    zwischenstandFiles = os.listdir(zwischenstandPath)
    if not zwischenstandFiles:
        df.to_pickle(MyPickleFile)
    else:
        for zwischenstandFile in zwischenstandFiles:
            zwischenstandFile = os.path.join(zwischenstandPath, zwischenstandFile)
            os.remove(zwischenstandFile)
        df.to_pickle(MyPickleFile)

In [ ]:
def splitFile(TRECfile, quantity):
    """
    TREC_Washington_Post_collection.v4.jl has 728626 Lines and has 15GB.
    It is to large to read into one Dataframe for the most PCs.
    With this function you can split the big TREC-File into as many
    small files you want.
    
    args: 
    TRECfile = Complete Path + Filename
    quantitiy = How many Small files you want to create
    """
    
    # Splitting the complete Path to Filename and Path-only
    TRECpath = TRECfile.split('\\')
    TRECpath = TRECfile[:(len(TRECpath[-1])*-1)-1]
    
    # Creating (if necessary) a subfolder 'small' for the new Files
    TRECpathSmall = os.path.join(TRECpath, 'small')
    if not os.path.exists(TRECpathSmall):
        os.makedirs(TRECpathSmall)
        print(f'{TRECpathSmall} wurde erstellt.')
    
    # If the folder 'small' already has splitted files: data skip the Split-function
    TRECfileSmall = os.listdir(TRECpathSmall)
    if not TRECfileSmall:
        smallfiles = []
        lines_per_file = int(729000/quantity)
        smallfile = None
        fileCounter = 1
        try:
            with open(TRECfile) as bigfile:
                for lineno, line in enumerate(bigfile):
                    if lineno % lines_per_file == 0:
                        if smallfile:
                            smallfile.close()
                        small_filename = f'TREC_Washington_Post_small_{fileCounter}.jl'
                        small_filename = os.path.join(TRECpathSmall, small_filename)
                        print(f'{small_filename} wird erstellt')
                        smallfiles.append(small_filename)
                        fileCounter += 1
                        smallfile = open(small_filename, "w")
                    smallfile.write(line)
                if smallfile:
                    smallfile.close()
            print(f'Es befinden {len(smallfiles)} JSON-Files im "small" Unterordner.')
            # return a list of the splitted data
            return smallfiles
        except:
            print('Path der TREC_Washington_Post_collection.v4.jl ist falsch\nDatei wurde nicht gefunden.')
    else:
        # If the folder 'small' is already there and has data create a list of the data and return it
        TRECfileSmall = os.listdir(TRECpathSmall)
        smallfiles = []
        for smallfile in TRECfileSmall:
            smallfile = os.path.join(TRECpathSmall, smallfile)
            smallfiles.append(smallfile)
        print(f'Es befinden bereits {len(smallfiles)} Files im "small" Unterordner.')
        return smallfiles

WpDataSmall = splitFile(myTRECfile, myWishedSmallFiles)

In [ ]:
def SmallFilesToPickle(listOfSmallFiles):
    """
    Opens the splitted JSON-Files and converts these dataframes
    into smaller sub-dataframes.
    
    args: 
    listOfSmallFiles = list of the splitted TREC File
    """
    fileending = listOfSmallFiles[0]
    fileending = pathlib.Path(fileending).suffix
    fileending = fileending[1:]
    # if fileending is .jl it can be read as a json
    if fileending == 'jl':
        listOfSmallFilesPickle = []
        for filename in listOfSmallFiles:
            df = pd.read_json(filename, lines=True)

            # Drop empty rows
            df.dropna(inplace = True)

            # delete duplicated and unnecessary columns
            if 'contents' in df.columns:
                df = df.drop(columns='contents')
            if 'type' in df.columns:
                df = df.drop(columns='type')
            if 'source' in df.columns:
                df = df.drop(columns='source')
            if 'orig-id' in df.columns:
                df = df.drop(columns='orig-id')
            if 'publish_date' in df.columns:
                df = df.drop(columns='publish_date')
                
            filenamemeameOld = filename
            fileNameNew = filename[:-3]+'.pickle'
            os.remove(filenamemeameOld)
            df.to_pickle(fileNameNew)
            listOfSmallFilesPickle.append(fileNameNew)
            del df
        print(f'Es wurden {len(listOfSmallFiles)} JSON-Files nach Pickle convertiert.')
        return listOfSmallFilesPickle
    # if fileending is .pickle it already has been split and the list just getting returned
    elif fileending == 'pickle':
        print(f'Es befinden {len(listOfSmallFiles)} Pickle-Files im "small" Unterordner.')
        return listOfSmallFiles
    else:
        # if fileending is an unusual one nothing happens
        print('Es befinden sich weder JSON- noch Pickle-Files im "small" Unterordner.')
        return None

WpDataSmall = SmallFilesToPickle(WpDataSmall)

In [ ]:
# ----------------------- LÖSCHEN WENN ANDERE FILES GENUTZT WERDEN SOLLEN  ------------------------

hundredthousandWPpath = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\hundredthousand_wp.pickle'

del WpDataSmall[:]
WpDataSmall.append(hundredthousandWPpath)
WpDataSmall

# --------------------------------------------------------------------------------------------------

In [ ]:
def createDfFromSmallFiles(listOfSmallFiles):
    """
    Opens all small-pickle-dataframes and connects them to one big Dataframe.
    
    args: 
    listOfSmallFiles = list of the splitted TREC File
    """
    fileending = listOfSmallFiles[0]
    fileending = pathlib.Path(fileending).suffix
    fileending = fileending[1:]
    if fileending == 'pickle':
        list_of_dataframes = []
        for filename in listOfSmallFiles:
            to_merge_df = pd.read_pickle(filename)
            list_of_dataframes.append(to_merge_df)

        # write all small dataframes in one big dataframe
        df = pd.concat(list_of_dataframes)        

        # delete the small dataframes for more memory
        del to_merge_df
        del list_of_dataframes

        return df

df = createDfFromSmallFiles(WpDataSmall)

# Randomize the rows and reset a new id
#df = df.sample(frac=1).reset_index(drop=True)

# !!! DATENSATZ VERKLEINERT
df = df[:myWishedDatasets]

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v1')

---
# | Cleaning & Transforming

In [ ]:
for column in df:
    df[column] = df[column].replace('', np.nan)
    df[column] = df[column].replace('None', np.nan)
    df[column] = df[column].replace(r'^\s*$', np.nan, regex=True)
df = df.fillna(value=np.nan)

In [ ]:
def dateconvert(x):
    try:
        import datetime
        import numpy as np   
        ts = datetime.datetime.fromtimestamp(x.published_date/1000)
        # year-month-day with hours-minutes-seconds for future analysis
        #date = ts.strftime('%Y-%m-%d %H:%M:%S')
        # year-month-day without time
        date = ts.strftime('%Y-%m-%d')
        return date
    except:
        return np.nan

# converts the epochs datetime to a normale datestring
df['published_date'] = df.parallel_apply(dateconvert, axis=1)

In [ ]:
def createText(x):
    import numpy as np
    try:
        import pandas as pd
        dfcontent = pd.json_normalize(x.content, max_level=1)
        dfcontent = dfcontent[dfcontent.subtype == 'paragraph']
        #dfcontent = dfcontent[dfcontent.mime == 'text/html']
        contentText = ''
        
        for index, row in dfcontent.iterrows():
            contentText += row["content"]
            contentText += ' '
            
            import re
            contentText = contentText.encode('utf-8').decode('ascii','ignore')
            contentText = re.sub('<[^<]+?>', '', contentText)
            contentText = contentText.replace(u'\xa0', u' ')
            contentText = contentText.replace(f'\\n', ' ')
            contentText = contentText.replace(f'\n', f' ')
            contentText = contentText.replace('  ',' ')
            
        return contentText
    except:
        return np.nan

# creates text from the content column by filtering html and json
df['text'] = df.parallel_apply(createText, axis=1)

# if content json is converted in text delete the column
df = df.drop(columns='content')

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v2')

In [ ]:
def combineText(x):
    import numpy as np
    try:
        title = x.title
        text = x.text
        if title == np.nan:
            title = ''
        if text == np.nan:
            title = ''
        mergedText = f'{title} {text}'
        return mergedText
    except:
        return np.nan

# write Title and Text in merged_total_text
df['merged_total_text'] = df.parallel_apply(combineText, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v3')

# | Gender Guesser

In [ ]:
#!pip install gender_guesser  

def authorGender(x):
    import numpy as np
    try:
        import gender_guesser.detector as gender
        gd = gender.Detector()
        author = x.author
        if author == np.nan:
            author = ''
        author = author.split(' ')[0]
        gender = gd.get_gender(author)
        return gender
    except:
        return np.nan

df['author_gender'] = df.parallel_apply(authorGender, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v4')

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v5')

In [ ]:
def findCategory(x):
    import numpy as np
    try:
        articleUrl = x.article_url

        if articleUrl[:5] == 'https':
            category = articleUrl.split('/')[3]
        elif articleUrl[:1] == '/':
            category = articleUrl.split('/')[1]
        else:
            category = 'other'
        return category
    except:
        return np.nan

# find category from article_url
df['category'] = df.parallel_apply(findCategory, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v6')

In [ ]:
def communion_context(x):
    import numpy as np
    try:
        merged_total_text = x.merged_total_text
        file = 'a_communion.txt'
        with open(file, "r") as tf:
            lines = tf.read().split('\n')
        import re
        counter = 0
        for line in lines:
            count = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(line), merged_total_text))
            counter += count
        return counter
    except:
        return np.nan

df['communion_context'] = df.parallel_apply(communion_context, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v7')

In [ ]:
def agency_context(x):
    import numpy as np
    try:
        merged_total_text = x.merged_total_text
        file = 'a_agency.txt'
        with open(file, "r") as tf:
            lines = tf.read().split('\n')
        import re
        counter = 0
        for line in lines:
            count = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(line), merged_total_text))
            counter += count
        return counter
    except:
        return np.nan

df['agency_context'] = df.parallel_apply(agency_context, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v8')

In [ ]:
#Text anstatt Titel
def getTextLenght(x):
    import numpy as np
    try:
        return len(x.text)
    except:
        return np.nan

# write length of title
df['len_text'] = df.parallel_apply(getTextLenght, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v9')

In [ ]:
#Text anstatt Titel
def getTextWordLenght(x):
    import numpy as np
    try:
        text = x.text
        import spacy
        nlp = spacy.load('en_core_web_sm')
        text = nlp(text)
        tokenizerLength = len(text)
        return tokenizerLength
    except:
        return np.nan

# write number of title words
df['word_text'] = df.parallel_apply(getTextWordLenght, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v10')

In [ ]:
#Text anstatt Titel
def TitleVaderScore(x):
    import numpy as np
    try:
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        analyzer = SentimentIntensityAnalyzer()
        text = x.text
        vader_score = analyzer.polarity_scores(text)['compound']
        if vader_score >= 0.05 : 
            return("Positive") 
        elif vader_score <= - 0.05 : 
            return("Negative")  
        else : 
            return("Neutral")
    except:
        return np.nan

# find category from article_url
df['compund'] = df.parallel_apply(TitleVaderScore, axis=1)

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v11')

# | Named Entity Recognition w/ spaCy NEU

In [ ]:
df_ner_full = df

In [ ]:
# write entity in parsed_text 
parsed_text = []
for text, id in tqdm.tqdm(zip(df_ner_full['merged_total_text'], df_ner_full['id']), total=len(df_ner_full)):
    parsed_text.extend([[id, str(entity), entity.label_] for entity in nlp(text).ents])

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v12')

In [ ]:
# create dfs from entities
df_ner = pd.DataFrame(parsed_text, columns=['id', 'merged_total_text', 'entity_type'])
df_ner.rename(columns={'merged_total_text': 'entity_per'}, inplace=True)

In [ ]:
# merge back to initial df
df_ner_full = (pd.merge(df_ner_full, df_ner, on='id')) 

In [ ]:
df_ner_full = (df_ner_full.loc[df_ner_full['entity_type'].isin(['PERSON'])])

In [ ]:
# Replace artist names (Sting, Bono, Cher etc.) with name according to wikipedia
# TO do: write into dict?
artists = {
            'JFK': 'John Fitzgerald Kennedy',
            'FDR': 'Franklin Delano Roosevelt',
            'Sting': 'Gordon Matthew Sumner',
            'Bono': 'Paul David Hewson',
            'Cher': 'Cherilyn Sarkisian',
            'Madonna': 'Madonna Louise Ciccone',
            'Adele': 'Adele Laurie Adkins',
            'Eminem': 'Marshall Bruce Mathers',
            'Beyonce': 'Beyoncé Knowles-Carter',
            'Blaine Friedlander': 'Blaine P. Friedlander Jr.'
        }
df_ner_full['entity_per'] = df_ner_full['entity_per'].replace(artists)

In [ ]:
def split_name(df, var):
    sub_df = df[var].str.split('\\s+', expand=True)
    result = []

    for _, row in sub_df.iterrows():
        info = {'first_name': '', 'middle_name': '', 'last_name': ''}
        n = row.count()

        if n == 0:
            pass
        elif n == 1:
            info['first_name'] = row.iloc[0]
        elif n == 2:
            info['first_name'], info['last_name'] = row.iloc[:2]
        else:
            info['first_name'] = row.iloc[0]
            info['last_name'] = row.iloc[-1]
            info['middle_name'] = ' '.join([(string or '') for string in row.iloc[1:-1]])
        result.append(info)
    return pd.DataFrame(result, index=df.index)

df_names = split_name(df_ner_full, 'entity_per')
df_ner_full = df_ner_full.join(df_names)

In [ ]:
#!pip install gender_guesser    
import gender_guesser.detector as gender
gd = gender.Detector()
df_ner_full['gender_guesser'] = df_ner_full['first_name'].apply(str.capitalize).map(lambda x: gd.get_gender(x))

In [ ]:
#df.drop(['agency_context', 'len_title', 'word_title', 'compund', 'content', 'communion_context', 'category' , 'entitys'], axis=1, inplace=True)#

In [ ]:
df_ner_full.gender_guesser.value_counts()

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v13')

In [ ]:
df_ner_full

# | Auswertung

In [ ]:
# Laden des aktuellen Stands
#aktuellPickle = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\aktuell.pickle'
#df = pd.read_pickle(aktuellPickle)
#df = df_ner_full

In [ ]:
df.groupby("author_gender")["id"].count()

In [ ]:
# Es wurden nur 280 AutorInnen ein Gender zugewiesen
df["author_gender"].count()

In [ ]:
# Insgesamt besteht der Datensatz aktuell aus 300 Zeilen
df["id"].count()

In [ ]:
gender = ["andy", "female", "male", "mostly_female", "mostly_male", "unknown"]
other = ["andy", "mostly_female", "mostly_male", "unknown"]

In [ ]:
# Artikel denen Gender "None" zugeordnet wurde
df[~df.author_gender.isin(gender)]["id"].count()

In [ ]:
# None Werte aus Spalte 'author_gender' durch 'unknown' ersetzen
df['author_gender'] = df['author_gender'].replace({None: 'unknown'})

In [ ]:
# Anzahl Artikel gesamt
'Im Datensatz sind ingesamt ' + str(df["id"].count()) + ' Artikel vorhanden'

In [ ]:
# Anzahl AutorenInnen
'Die ' + str(df["id"].count()) + ' Artikel wurden von insgesamt ' + str(df["author"].nunique()) + ' AutorInnen verfasst'

In [ ]:
# Anzahl AutorenInnen nach Gender
df.groupby('author_gender')["id"].count().sort_values(ascending=False)

In [ ]:
df.groupby('author_gender')["id"].count().sort_values(ascending=False).plot(kind='bar')
plt.title('Anzahl AutorInnen nach Geschlecht')
plt.xlabel('Geschlecht')
plt.ylabel('Anzahl Artikel')

In [ ]:
'Die Grafik zeigt, dass ' \
+ str(df[df["author_gender"] == 'male']["id"].count()) + ' Artikel von Männern, ' \
+ str(df[df["author_gender"] == 'female']["id"].count()) + ' von Frauen und ' \
+ str(df[df["author_gender"].isin(other)]["id"].count()) + ' vom Gender Guesser nicht klar zuordenbaren Geschlechtern verfasst worden sind.'

In [ ]:
# Anzahl Artikel pro AutorIn nach Gender
'Männliche Autoren verfassen bei der Washington Post im Durchschnitt ' + str(round(df[df["author_gender"] == 'male'].groupby("author")["id"].count().mean(), 2)) + ' Artikel, ' \
+ 'Frauen hingegen im Durschnitt ' + str(round(df[df["author_gender"] == 'female'].groupby("author")["id"].count().mean(), 2)) + ' Artikel'

In [ ]:
# Vader Stimmungsanalyse Auswertung (absolut) - Männliche Autoren
df[df["author_gender"] == 'male'].groupby(['compund'])["compund"].count().sort_values(ascending=False).plot(kind='bar')
plt.title('Auswertung Stimmungsanalyse bei Artikeln von männlichen Autoren (absolut)')
plt.xlabel('Stimmung')
plt.ylabel('Anzahl Artikel')
plt.ylim([10, 80])

In [ ]:
# Vader Stimmungsanalyse Auswertung (relativ) - Männliche Autoren
vader_male = pd.DataFrame(df[df["author_gender"] == 'male'].groupby(['compund'])["id"].count().sort_values(ascending=False))
vader_male["total"] = df[df["author_gender"] == 'male']["id"].count()
vader_male["relative"] = vader_male["id"] / vader_male["total"] * 100
vader_male["relative"].sort_values(ascending=False).plot(kind='pie', autopct='%1.1f%%')
plt.title('Auswertung Stimmungsanalyse bei Artikeln von männlichen Autoren (relativ)')
plt.xlabel("")
plt.ylabel("")

In [ ]:
# Vader Stimmungsanalyse Auswertung (absolut) - Weibliche Autorinnen
df[df["author_gender"] == 'female'].groupby(['compund'])["compund"].count().sort_values(ascending=False).plot(kind='bar')
plt.title('Auswertung Stimmungsanalyse bei Artikeln von weiblichen Autorinnen (absolut)')
plt.xlabel('Stimmung')
plt.ylabel('Anzahl Artikel')
plt.ylim([10, 80])

In [ ]:
# Vader Stimmungsanalyse Auswertung (relativ) - Weibliche Autorinnen
vader_female = pd.DataFrame(df[df["author_gender"] == 'female'].groupby(['compund'])["id"].count().sort_values(ascending=False))
vader_female["total"] = df[df["author_gender"] == 'female']["id"].count()
vader_female["relative"] = vader_female["id"] / vader_female["total"] * 100
vader_female["relative"].sort_values(ascending=False).plot(kind='pie', autopct='%1.1f%%')
plt.title('Auswertung Stimmungsanalyse bei Artikeln von weibliche Autorinnen (relativ)')
plt.xlabel("")
plt.ylabel("")

In [ ]:
# Vader Stimmungsanalyse Auswertung (relativ) - AutorInnen im Vergleich (Datentabelle)
vader_male = vader_male.rename(columns={"id": "vader_male_count", "total": "vader_male_total", "relative": "vader_male_relative"})
vader_female = vader_female.rename(columns={"id": "vader_female_count", "total": "vader_female_total", "relative": "vader_female_relative"})
vader_total = pd.concat([vader_male,vader_female], axis = 1, sort = False)
vader_total

In [ ]:
# Vader Stimmungsanalyse Auswertung (relativ) - AutorInnen im Vergleich (Plot)
vader_total[["vader_male_relative", "vader_female_relative"]].plot(kind='bar')
plt.title('Auswertung Stimmungsanalyse bei Artikeln von AutorInnen (relativ)')
plt.xlabel('Stimmung')
plt.ylabel('Anteil Artikel (in%)')

In [ ]:
# Top 5 Kateogorien (absolut) - Männliche Autoren
df[df.author_gender == 'male'].groupby(['category'])["id"].count().sort_values(ascending=False).head(5).plot(kind='bar')
plt.title('Top 5 Kategorien von männlichen Autoren (absolut)')
plt.xlabel('Kategorie')
plt.ylabel('Anzahl Artikel')

In [ ]:
# Top 5 Kateogorien (relativ) - Männliche Autoren
top_male = pd.DataFrame(df[df.author_gender == 'male'].groupby(['category'])["id"].count().sort_values(ascending=False).head(5))
top_male["total"] = df[df["author_gender"] == 'male']["id"].count()
top_male["relative"] = top_male["id"] / top_male["total"] * 100
top_male["relative"].sort_values(ascending=False).plot(kind='pie', autopct='%1.1f%%')
plt.title('Top 5 Kategorien von männlichen Autoren (relativ)')
plt.xlabel("")
plt.ylabel("")

# Info: Prozentualer Anteil auf Basis der Top 5 Kategorien

In [ ]:
# Top 5 Kateogorien - Weibliche Autorinnen
df[df.author_gender == 'female'].groupby(['category'])["id"].count().sort_values(ascending=False).head(5).plot(kind='bar')
plt.title('Top 5 Kategorien von weiblichen Autorinnen (absolut)')
plt.xlabel('Kategorie')
plt.ylabel('Anzahl Artikel')

In [ ]:
# Top 5 Kateogorien (relativ) - Weibliche Autoreninnen
top_female = pd.DataFrame(df[df.author_gender == 'female'].groupby(['category'])["id"].count().sort_values(ascending=False).head(5))
top_female["total"] = df[df["author_gender"] == 'female']["id"].count()
top_female["relative"] = top_female["id"] / top_female["total"] * 100
top_female["relative"].sort_values(ascending=False).plot(kind='pie', autopct='%1.1f%%')
plt.title('Top 5 Kategorien von weiblichen Autoreninnen (relativ)')
plt.xlabel("")
plt.ylabel("")

# Info: Prozentualer Anteil auf Basis der Top 5 Kategorien

In [ ]:
# Auswertung Länge der Artikel/Texte und Speicherung unter neuer Spalte 'len_text'
df["len_text"] = df.apply(lambda row: len(str(row["text"])), axis=1)

In [ ]:
# Durchschnittliche Wörter nach Geschlecht
df[~df.author_gender.isin(other)].groupby(['author_gender'])["len_text"].mean().plot(kind='barh')
plt.title('Durschnittliche Anzahl von Wörtern von Washington Post Artikeln')
plt.xlabel('Geschlecht')
plt.ylabel('Durschnittliche Anzahl Wörter')

In [ ]:
'Die Grafik zeigt, dass' \
+ ' männliche Autoren im Durchschnitt ' +  str(round(df[df["author_gender"] == 'male']["len_text"].mean(),2)) + ' Wörter verfassen' \
+ ' und weibliche Autorinnen im Durchschnitt ' +  str(round(df[df["author_gender"] == 'female']["len_text"].mean(),2)) + ' Wörter verfassen '

### Gender Guesser

In [ ]:
# Anzahl von Gender-Erwähnungen in Artikeln pro Author Gender
df_ner_full[df_ner_full.author_gender == 'male'].groupby(['gender_guesser'])["gender_guesser"].count().sort_values(ascending=False).plot(kind="bar")

In [ ]:
# Anzahl von Gender-Erwähnungen in Artikeln pro Author Gender
df_ner_full[df_ner_full.author_gender == 'female'].groupby(['gender_guesser'])["gender_guesser"].count().sort_values(ascending=False).plot(kind="bar")

In [ ]:
male_guesser = pd.DataFrame(df_ner_full[df_ner_full.author_gender == 'male'].groupby(['gender_guesser'])["id"].count())
male_guesser = male_guesser.rename(columns={"id": "count_male"})
female_guesser = pd.DataFrame(df_ner_full[df_ner_full.author_gender == 'female'].groupby(['gender_guesser'])["id"].count())
female_guesser = female_guesser.rename(columns={"id": "count_female"})
total_guesser = pd.concat([male_guesser,female_guesser], axis = 1, sort = False)
total_guesser

In [ ]:
total_guesser = total_guesser.drop(other)
total_guesser

In [ ]:
total_guesser["total_male"] = df_ner_full[df_ner_full.author_gender == 'male']["id"].count()
total_guesser["relative_male"] = total_guesser["count_male"] / total_guesser["total_male"] * 100
total_guesser["total_female"] = df_ner_full[df_ner_full.author_gender == 'female']["id"].count()
total_guesser["relative_female"] = total_guesser["count_female"] / total_guesser["total_female"] * 100
total_guesser

In [ ]:
total_guesser[["count_male", "count_female"]].sort_values(by='count_male', ascending=False).plot(kind="bar")
plt.title('Identifizierung von erwähnten Personen in Artikeln (absolut)')
plt.xlabel('Geschlecht')
plt.ylabel('Anzahl')

In [ ]:
total_guesser[["relative_male", "relative_female"]].sort_values(by='relative_male', ascending=False).plot(kind="pie", subplots=True, autopct='%1.1f%%')
plt.title('Identifizierung von erwähnten Personen in Artikeln (relative)')

In [ ]:
print("Sum Communion Male:" + str(df[df["author_gender"] == 'male']["communion_context"].sum()))
print("Sum Agency Male:" + str(df[df["author_gender"] == 'male']["agency_context"].sum()))

print("Sum Communion Female:" + str(df[df["author_gender"] == 'female']["communion_context"].count()))
print("Sum Agency Female:" + str(df[df["author_gender"] == 'female']["agency_context"].count()))

In [ ]:

print("Relativer Anteiler Communion Male:" + str(df[df["author_gender"] == 'male']["communion_context"].sum() / (df[df["author_gender"] == 'male']["communion_context"].sum() + df[df["author_gender"] == 'male']["agency_context"].sum()) * 100))
print("Relativer Anteiler Agency Male:" + str(df[df["author_gender"] == 'male']["agency_context"].sum() / (df[df["author_gender"] == 'male']["communion_context"].sum() + df[df["author_gender"] == 'male']["agency_context"].sum()) * 100))

print("Relativer Anteiler Communion Female:" + str(df[df["author_gender"] == 'female']["communion_context"].sum() / (df[df["author_gender"] == 'female']["communion_context"].sum() + df[df["author_gender"] == 'female']["agency_context"].sum()) * 100))
print("Relativer Anteiler Agency Female:" + str(df[df["author_gender"] == 'female']["agency_context"].sum() / (df[df["author_gender"] == 'female']["communion_context"].sum() + df[df["author_gender"] == 'female']["agency_context"].sum()) * 100))

In [ ]:
# Gründe für Unkown aufzeigen
# Was ist wenn mehrere Persons in Artikel? Frage an Julia
# Gender None , warum?

In [ ]:
if myZwischenSpeichern:
    zwischenSpeichern(myDataPath, 'v14')

In [ ]:
zeitende = time.time()
zeitdauer = zeitende-zeitanfang
zeitdauer = time.strftime('%H:%M:%S', time.gmtime(zeitdauer))

print(f'Dauer Programmausführung:\n{zeitdauer}\nbei {myWishedDatasets} Datensätzen ----------------------------------------')

## Snorkel

In [ ]:
!pip install snorkel

In [ ]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

In [ ]:
shuffled = df.sample(frac=1)

In [ ]:
df_train,df_test = train_test_split(df, train_size=0.7)
print(df_train.shape)

#### Regeln für Unknown in Autor:

Männer schreiben mehr über Sports und Politics

Frauen schreiben mehr über Lifestyle und Blogs

Texte von Männern sind länger als die von Frauen

In [ ]:
MALE_AUTHOR = 1
FEMALE_AUTHOR = -1
UNIDENTIFIED_AUTHOR = 0

In [ ]:
@labeling_function()
def lf_category_lookup(x):
    keywords_male = ["sports", "politics"]
    keywords_female = ["blogs", "lifestyle"]
    if any(word in x.category.lower() for word in keywords_male):
        return MALE_AUTHOR
    elif any(word in x.category.lower() for word in keywords_female):
        return FEMALE_AUTHOR
    else:
        return UNIDENTIFIED_AUTHOR

In [ ]:
@labeling_function()
def lf_word_lookup(x):
    import numpy as np
    try:
        text = x.text
        import spacy
        nlp = spacy.load('en_core_web_sm')
        text = nlp(text)
        tokenizerLength = len(text)
        if tokenizerLength >= 3900:
            return MALE_AUTHOR
        else:
            return FEMALE_AUTHOR
    except:
        return np.nan   

In [ ]:
lfs = [lf_category_lookup,lf_word_lookup]

In [ ]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

In [ ]:
#Erst aufrufen, wenn wir eine dritte Funktion zur Unterscheidung haben
label_model = LabelModel(cardinality=2,verbose=True)
#label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)

In [ ]:
df_train['Labels_Author'] = label_model.predict(L=L_train,tie_break_policy="unindetified")

#### Regeln für Gender innerhalb der Texte:

Männer werden mehr mit Agency Wörtern beschrieben

Frauen werden mit mehr Community Wörtern beschrieben

"she" ist weiblich, "he" ist männlich

In [ ]:
MALE_TEXT = 1
FEMALE_TEXT = -1
UNIDENTIFIED_TEXT = 0

In [ ]:
@labeling_function()
def lf_prenoms_lookup(x):
    keywords_male = ["he", "his", "him"]
    keywords_female = ["he", "his", "him"]
    if any(word in x.text.lower() for word in keywords_male):
        return MALE_TEXT
    elif any(word in x.text.lower() for word in keywords_female):
        return FEMALE_TEXT
    else:
        return UNIDENTIFIED_TEXT

In [ ]:
'''
lfs_text = [lf_prenoms_lookup]
applier_text = PandasLFApplier(lfs=lfs_text)
L_train = applier.apply(df=df_train)
label_model = LabelModel(cardinality=2,verbose=True)
#label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)
df_train['Labels_Persons'] = label_model.predict(L=L_train,tie_break_policy="unindetified")
'''

---
---
---
---
---
---

# | UNFERTIGE FUNKTIONEN

In [ ]:
# id
# article_url
# title
# author
# published_date
# type
# source
# content
# publish_date
# orig-id
# text
# merged_total_text
# entitys

# df[df['publish_date'].isnull()]

def createEntitys(x):
    import spacy
    import en_core_web_trf

    # TRF Orininal / Md schneller
    nlp = spacy.load("en_core_web_md")
    # JULIA nlp = spacy.load("en_core_web_trf")
    
    totalText = x['merged_total_text']
    try:
        entitys = {}
        for word in nlp(totalText).ents:
        # WENN WORD.LABEL_ IS PERSON DANN NUR HINZUFÜGEN?!
            entitys[str(word)] = word.label_
        return entitys
    except:
        return np.nan

#x = df.entitys[0]
#for word, entity in x.items():
#    if entity == 'PERSON':
#        print(f'{word} is {entity}')

#df_ner_full.head(100)


# WIRD IN DER ANALYSE NICHT BENÖTIGT
# df['entitys'] = df.parallel_apply(createEntitys, axis=1)

In [ ]:
# NER 'PERSON' STARTS HERE
#df = (df.loc[df['entity_type'].isin(['PERSON'])])

In [ ]:
#df["entity"] = df['entity'].str.replace('[\[\]\"\'\d\,\<\/]','')

In [ ]:
def createEntitys(x):
    import spacy
    import en_core_web_trf

    # TRF Orininal / Md schneller
    nlp = spacy.load("en_core_web_md")
    # JULIA nlp = spacy.load("en_core_web_trf")
    
    totalText = x['merged_total_text']
    try:
        entitys = {}
        for word in nlp(totalText).ents:
        # WENN WORD.LABEL_ IS PERSON DANN NUR HINZUFÜGEN?!
            entitys[str(word)] = word.label_
        return entitys
    except:
        return None

#x = df.entitys[0]
#for word, entity in x.items():
#    if entity == 'PERSON':
#        print(f'{word} is {entity}')

#df_ner_full.head(100)


# WIRD IN DER ANALYSE NICHT BENÖTIGT
# df['entitys'] = df.parallel_apply(createEntitys, axis=1)

In [ ]:
# Replace artist names (Sting, Bono, Cher etc.) with name according to wikipedia
def replaceEntityNames(x):
    try:
        entitys = x.entitys
        artists = {
            'JFK': 'John Fitzgerald Kennedy',
            'FDR': 'Franklin Delano Roosevelt',
            'Sting': 'Gordon Matthew Sumner',
            'Bono': 'Paul David Hewson',
            'Cher': 'Cherilyn Sarkisian',
            'Madonna': 'Madonna Louise Ciccone',
            'Adele': 'Adele Laurie Adkins',
            'Eminem': 'Marshall Bruce Mathers',
            'Beyonce': 'Beyoncé Knowles-Carter',
            'Blaine Friedlander': 'Blaine P. Friedlander Jr.'
        }

        for entityKey, entityValue in entitys.items():
            if entityValue == 'PERSON':
                for artAbkzg, artName in artists.items():
                    if entityKey == artAbkzg:
                        entitys[artName]=entityValue
                        del entitys[entityKey]  
    
        return entitys
    except:
        return x.entitys
# !!! NOCH FEHLERHAFT
#df['entitys'] = df.parallel_apply(replaceEntityNames, axis=1)

In [ ]:
#df = df[df['author'] != df['entity']] # if author is in col entity -> remove, we only want "Person" from within the article
# To do: Blaine P. Friedlander Jr. vs. Blaine Friedlander 

## Split first, middle and last name in different cols

In [ ]:
# Write first, middle and last name in different cols
# to do: split middle and last name does not work
def split_name(df, var):
    sub_df = df[var].str.split('\\s+', expand=True)
    result = []

    for _, row in sub_df.iterrows():
        info = {'first_name': '', 'middle_name': '', 'last_name': ''}
        n = row.count()

        if n == 0:
            pass
        elif n == 1:
            info['last_name'] = row.iloc[0]
        elif n == 2:
            info['first_name'], info['last_name'] = row.iloc[:2]
        else:
            info['first_name'] = row.iloc[0]
            info['last_name'] = row.iloc[-1]
            info['middle_name'] = ' '.join([(string or '') for string in row.iloc[1:-1]])
        result.append(info)
    return pd.DataFrame(result, index=df.index)

df_names = split_name(df, 'entitys')
df = df.join(df_names)

## | Enrich first name if only last name is mentioned

In [ ]:
names = df.groupby('id')['entity'].apply(lambda x: list(np.unique(x)))
# To do: if only one token ("Obama") drop or don't write to list

In [ ]:
# function for enriching first name if only last name is mentioned
from tqdm import tqdm
tqdm.pandas()
def enrich_firstname(row):
    
    entity = row['entity']#.copy()
    firstname = row['first_name']#.copy()
    lastname = row['last_name']#.copy()
    
    #if len(firstname) == 0:
        # look in list per id for key and write value?
    # elif 
    # bestehende Vornamen nicht überschreiben
    #else:
    #    row['first_name'] = 'unknown'
    return row

df_test = df_test.progress_apply(enrich_firstname, axis=1)


# | Gender Guesser

In [ ]:
# !pip install gender_guesser    
import gender_guesser.detector as gender
gd = gender.Detector()
df['gender_guesser'] = df['first_name'].apply(str.capitalize).map(lambda x: gd.get_gender(x))

> __unknown__ (name not found), __andy__ (androgynous), __male__, __female__, __mostly_male__, or __mostly_female__. The difference between andy and unknown is that the former is found to have the same probability to be male than to be female, while the later means that the name wasn’t found in the database.

In [ ]:
df.gender_guesser.value_counts()

In [ ]:
df.loc[df['gender_guesser'] == 'unknown'][:40]

In [ ]:
# df.first_name.value_counts()[:40]

In [ ]:
# to do:
# Namen cleanen
# weitere lib für gender identification einbauen
# function schreiben: wenn nur Nachname genannt, schaue in Liste/anderer Zeile pro Artikel, ob Name schon genannt, dann Vorname auffüllen
# oder
# wenn Obama, Santorum, Gingrich, etc. dann aus erstellter Liste oder via Wikidata mit Vornamen auffüllen
# # dominique rodgers cromartie etc. (famous people)  -> Wikidata? 

---
# | Group per article and get share of gender per article

In [ ]:
# next

# | Wikidata Query

In [ ]:
# oder das https://stackoverflow.com/questions/51419785/extract-data-from-wikidata-in-python

import requests

sparql_query = """
        prefix schema: <http://schema.org/>
        SELECT ?item ?occupation ?genderLabel ?bdayLabel
        WHERE {
            <https://en.wikipedia.org/wiki/Angela_Merkel> schema:about ?item .
            ?item wdt:P21 ?gender .
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        }
    """

In [ ]:
url = 'https://query.wikidata.org/sparql'

# sleep(2)
r = requests.get(url, params={'format': 'json', 'query': sparql_query})

In [ ]:
url = 'https://query.wikidata.org/sparql'

r = requests.get(url, params={'format': 'json', 'query': sparql_query})
data = r.json()

print(data['results']['bindings'])